In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyneb as pn

%matplotlib inline

In [ ]:
def load_CIE_data():
    # Collisional ionisation equilibrium (CIE) abundances from Gnat & Sternberg (2007, ApJS, 168, 213)
    CIE_data = np.loadtxt("CIE-abundances-gs07.txt")
    return CIE_data

def get_ion_abundance(CIE_data, ion, T):
    ion_list = dict([('HI',  1),  ('HII',  2), 
                     ('HeI', 3),  ('HeII', 4),  ('HeIII', 5),
                     ('NI',  13), ('NII',  14), 
                     ('OI',  21), ('OII',  22), ('OIII',  23), 
                     ('SI',  69), ('SII',  70), ('SIII',  71)])
    
    ion_index = ion_list[ion]
    ion_temps = data[:,0]
    T_index = np.searchsorted(ion_temps, T)
    abundance = np.zeros(T.size)
    dT = np.zeros(T.size)

    # If the requested temperature is outside of the tabulated range, we return the first or last value from the table, as appropriate
    # Otherwise, we calculate the value using linear interpolation
    
    mask1 = (T_index >= ion_temps.size)
    mask2 = (T_index == 0)
    mask3 = (~mask1 & ~mask2)

    abundance[mask1] = data[-1, ion_index]
    abundance[mask2] = data[0,  ion_index]
    dT[mask3] = ion_temps[T_index[mask3]] - ion_temps[T_index[mask3]-1]
    abundance[mask3] = data[T_index[mask3]-1,ion_index] + (data[T_index[mask3],ion_index] - data[T_index[mask3]-1,ion_index]) * ((T[mask3] - ion_temps[T_index[mask3]-1]) / dT[mask3])
    
    return abundance

def compute_emissivity(CIE_data, ion, transition, density, T):
    # Current we assume we're dealing with solar metallicty gas. Could be modified to take individual elemental
    # abundances or total metallicity as an input parameter, but that doesn't seem necessary right now
    solar_abundances = dict([('H', 1.0), ('He', 0.1), ('N', 6.03e-5), ('O', 4.57e-4), ('S', 1.38e-5)])

    # Check whether the transition wavelength we've passed in is one we're expecting to see
    # Note: if the value is a long way off from what it expects, PyNeb will also produce an error
    known_transitions = (3726, 3729, 5007, 6300, 6583, 6717, 6731, 9069, 9531) 
    if transition not in known_transitions:
        print("Unexpected transition wavelength: ", transition)
    
    if ion[0:2] == 'He':
        element = 'He'
        charge_state = len(ion) - 2
    else:
        element = ion[0]
        charge_state = len(ion) - 1  

    # Get number density of emitting ion
    elemental_abundance = solar_abundances[element]
    ion_fraction = get_ion_abundance(data, ion, T)
    ion_density = elemental_abundance * ion_fraction * density

    # Get electron number density. Currently, we also assume CIE here but we could alternatively extract this from the FLASH
    # simulation snapshots and pass it in as an additional parameter.
    #
    # Note: we include contribution from helium here, but not from metals -- this is a <1% effect, hence unlikely to be important
    electron_fraction = get_ion_abundance(data, 'HII', T) + 0.1 * get_ion_abundance(data, 'HeII', T) + 0.2 * get_ion_abundance(data, 'HeIII', T)
    electron_density = electron_fraction * density

    atom = pn.Atom(element, charge_state)
    
    # This computes the emission coefficient (units: erg cm^3 s^-1) for the transition from the specified ion at the specified wavelength
    # To compute the emissivity (units: erg cm^-3 s^-1), we then simply multiply by the ion density and the electron density
    emission_coeff = atom.getEmissivity(tem=T, den=density, wave=transition)
    emissivity = emission_coeff * ion_density * electron_density
    
    return emissivity

In [ ]:
# Usage example
data = load_CIE_data()

# 'density' is the number density of H nuclei
# For solar metallicity gas, this is related to the mass density rho via:  rho = 1.4 * mH * density,
# where mH is the mass of an H nucleus. [Note: the factor of 1.4 accounts for helium]
# 
# For this example, we use a fixed density and a small range of temperatures, but for post-processing
# the simulations, you'll obviously use the values derived from the snapshot
density = 100
output_T     = np.logspace(4, 6, 1000)

# Emissivity of the two lines in the [SII] doublet, at 6717 and 6731 Angstrom
emissivity_S1 = compute_emissivity(data, 'SII',  6717, density, output_T)
emissivity_S2 = compute_emissivity(data, 'SII',  6731, density, output_T)

# Total emissivity of the doublet
emissivity_S = emissivity_S1 + emissivity_S2

# This is just one of the lines in the [OII] doublet 
emissivity_O2 = compute_emissivity(data, 'OII', 3726, density, output_T)

# The [OIII] transition at 5007 Angstrom
emissivity_O3 = compute_emissivity(data, 'OIII', 5007, density, output_T)

plt.yscale("log")
plt.xscale("log")
plt.xlabel("Temperature [K]")
plt.ylabel(r"Emissivity [${\rm erg \: cm^{-3} \: s^{-1}}$]")
plt.plot(output_T, emissivity_S)
plt.plot(output_T, emissivity_O2)
plt.plot(output_T, emissivity_O3)